In [1]:
# Load Raw Data and URL category
import numpy as np
import pandas as pd
from IPython.display import display
from urlparse import urlparse
import datetime

data = pd.read_csv("hackathon_prepost.csv")
data_url = pd.read_csv("url_category.csv")


In [2]:
# Parse url data
temp = []
i = 0
for add in data['url']:
#    i = i+1
    if pd.notnull(add):
        temp1 = urlparse(add)     
        if temp1.netloc == '':
            strtmp = temp1.path
            strtmp = strtmp.split('&', 1)[0]
            temp.append(strtmp)
        else:
            temp.append(temp1.netloc)
    else:
        temp.append('NaN')
#    if i>8000000:
#        break
#display (len(temp),temp[:10])
data['URL'] = temp


In [3]:
display(len(data['url']))

1439613

In [4]:
#Merge url category data and raw data
data = pd.merge(data, data_url, on='URL', how='left')

In [6]:
# Convert Unix time to Thailand Time

import datetime
TimeSlot = []

for row in data['created_at']:
   t = int(datetime.datetime.utcfromtimestamp(int(row)/1000).strftime('%H'))+7
   if t<=7: 
       t = 'Night'
   elif t>7 and t<=18:
       t = 'Day'
   else: 
       t = 'Evening'
   TimeSlot.append(t)

#print len(TimeSlot) 
data['timeslot'] = TimeSlot

In [7]:
# Define Weekend and Weekdays

data['period'] = data[['year', 'month','day']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
data['dates'] = pd.to_datetime((data['period']), format="%Y %m %d")
data['dayofweek'] = data['dates'].dt.dayofweek
weekend = []
for row in data['dayofweek']:
    if row == 6 or row == 5:
            weekend.append('WEEKEND')
    else:
            weekend.append('WEEKDAY')
            
data['weekend'] = weekend

In [8]:
# One hot encoding of url type time slot, carrier,weekend
url = pd.get_dummies(data['URL_TYPE'])
url = pd.DataFrame(data= url)
weekend_encode = pd.get_dummies(data['weekend'])
weekend_encode = pd.DataFrame(data= weekend_encode)
timeslot_encode = pd.get_dummies(data['timeslot'])
timeslot_encode = pd.DataFrame(data= timeslot_encode)
carrier_encode = pd.get_dummies(data['carrier'])
carrier_encode = pd.DataFrame(data= carrier_encode)
data = data.join(url)
data = data.join(timeslot_encode)
data = data.join(weekend_encode)
data = data.join(carrier_encode)

In [9]:
# Merge other carrier information
data['others'] = data['TOT'] + data['True Move'] + data['AIS']

In [10]:
# Data agreegation on Tapad ID
Agg_data = data.groupby(['tapad_id'],as_index=False).agg({'AdTech':sum, 'Adult':sum, 'Unclassified':sum, 'eCommerce':sum, 'Education':sum, 'Entertainment':sum, 'Fashion':sum, 'Financial':sum, 'Health':sum, 'Info':sum, 'Job':sum,'Lottery':sum,'Music':sum, 'News':sum, 'Real Estate':sum, 'Social':sum, 'Sports':sum, 'Travel':sum,'Day':sum, 'Evening':sum,'Night':sum, 'WEEKDAY':sum,'WEEKEND':sum, 'DTAC':sum, 'WIFI':sum,'others':sum})

In [11]:
freq = data.groupby(['tapad_id']).size()
Agg_data['FREQ_CNT_TOTAL'] = freq
Agg_data= Agg_data.reset_index(drop=True)

In [12]:
data = data.sort_values(['year', 'month', 'day'], ascending=[False,False, False])
data_new = data.drop_duplicates(subset='tapad_id', keep="first")
data_new = data_new.reset_index(drop=True)
#data_new = data_new.rename(columns = {'month':'LATEST_MONTH','display_height':'LAST_DISPLAY_HEIGHT',
#'display_width':'LAST_DISPLAY_WIDTH','year_released':'YEAR RELEASED','diagonal_screen_size':'LAST_SCREEN_SIZE'})
#display(data_new.head(n=10))
#data_dim = data_new[['tapad_id', 'LATEST_MONTH','LAST_DISPLAY_HEIGHT','LAST_DISPLAY_WIDTH','YEAR RELEASED','LAST_SCREEN_SIZE']]

In [13]:
#final_data = Agg_data.join(data_, on='tapad_id')

final_data = pd.merge(Agg_data, data_new, on='tapad_id')


#olumns = final_data['Education_x','created_at','year','day','ip_address','carrier','user_agent','vendor','url','referrer_url','action_id','URL','CNT','URL_TYPE','timeslot','period','dates','dayofweek','weekend','AdTech_y','Adult_y','Education_y','Entertainment_y',
#'Fashion_y','Financial_y','Health_y','Info_y','Job_y','Lottery_y','Music_y','News_y','Real Estate_y','Social_y','Sports_y','Travel_y','Unclassified_y','eCommerce_y',
#'Day_y','Evening_y','Night_y','WEEKDAY_y','WEEKEND_y','AIS','DTAC_y','TOT','True Move','WIFI_y','others_y']
#data.drop(columns, inplace=True, axis=1)

In [14]:
final_data = final_data.rename(columns = {'tapad_id':'TAPAD_ID',
 'Lottery_x':'URL_LOTTERY',
 'Travel_x':'URL_TRAVEL',
 'Social_x':'URL_SOCIAL',
 'AdTech_x':'URL_ADTECH',
 'Sports_x':'URL_SPORTS',
 'Music_x':'URL_MUSIC',
 'WEEKDAY_x':'FREQ_CNT_WEEKDAY',
 'DTAC_x':'FREQ_CNT_DTAC',
 'Evening_x':'FREQ_CNT_EVENING',
 'Job_x':'URL_JOB',
 'Adult_x':'URL_ADULT',
 'News_x':'URL_NEWS',
 'WEEKEND_x':'FREQ_CNT_WEEKEND',
 'Day_x':'FREQ_CNT_DAY',
 'Info_x':'URL_INFO',
 'Unclassified_x':'URL_UNCLASSIFIED',
 'Night_x':'FREQ_CNT_NIGHT',
 'Fashion_x':'URL_FASHION',
 'Financial_x':'URL_FINANCIAL',
 'WIFI_x':'FREQ_CNT_WIFI',
 'eCommerce_x':'URL_ECOMMERCE',
 'Real Estate_x':'URL_REAL_E',
 'Health_x':'URL_HEALTH',
 'others_x':'FREQ_CNT_OTHER',
 'FREQ_CNT_TOTAL':'FREQ_CNT_TOTAL',
 'month':'LATEST_MONTH',
 'platform':'PLATFORM',
 'marketing_name':'MARKETING_NAME',
 'model':'MODEL',
 'display_height':'LAST_DISPLAY_HEIGHT',
 'display_width':'LAST_DISPLAY_WIDTH',
 'camera_pixels':'CAMERA',
 'year_released':'YEAR RELEASED',
 'diagonal_screen_size':'LAST_SCREEN_SIZE',
 'acct_type_cd':'ACCT_TYPE_CD'
 })
#final_data=final_data.reset_index(drop=True)
#display(data_new.head(n=10))

In [15]:
data_export = final_data[['TAPAD_ID','URL_LOTTERY','URL_TRAVEL','URL_SOCIAL','URL_ADTECH','URL_SPORTS','URL_MUSIC','FREQ_CNT_WEEKDAY','FREQ_CNT_DTAC',
 'FREQ_CNT_EVENING','URL_JOB','URL_ADULT','URL_NEWS','FREQ_CNT_WEEKEND','FREQ_CNT_DAY','URL_INFO','URL_UNCLASSIFIED','FREQ_CNT_NIGHT','URL_FASHION',
 'URL_FINANCIAL','FREQ_CNT_WIFI','URL_ECOMMERCE','URL_REAL_E','URL_HEALTH','FREQ_CNT_OTHER','FREQ_CNT_TOTAL','LATEST_MONTH','PLATFORM','MARKETING_NAME','MODEL',
 'LAST_DISPLAY_HEIGHT','LAST_DISPLAY_WIDTH','CAMERA','YEAR RELEASED','LAST_SCREEN_SIZE','ACCT_TYPE_CD']]

In [16]:
data_export

TAPAD_ID  URL_LOTTERY  \
0      0000bd8099979dca34a34809cfbe538786417792901c97...            0   
1      00038c601db2fc8d85cfe7a2dc708c68ee2cf9f1dc42b1...            1   
2      0005900f7d4c0c9c2a4da9e9a26109f03ba7deeee9ba4e...            0   
3      000627ef60a8f77ca11f734ba79ffbc5a4020f46cc51b2...            0   
4      000675789bbd3b0174a14d3d816eaa61212fd25b4fc2ec...            0   
5      0009e37d33f94b813fb04bc49186211d83773bda167737...            0   
6      000c5610f0d3ae3a23fe95b269a27644049ad5c6f76a52...            0   
7      001123f2c09fe89588fa290c192d6f2ba8969d5ee1c0a6...            0   
8      0013a52209758fd13d67e23a87a040eb11934ce08ba4c4...            0   
9      00148e44609fe82a430840c3cc35d035337e26c97b8a46...            0   
10     00149ebda679efa4a6b67c2092891e57324ecaaba730aa...            0   
11     00152c05042a4eadd3e9802c6131185fa7edd2d8eb6419...            0   
12     0015a5477fdd311ce6aeed51c4a5c5954d3439261ffc31...            0   
13     00173e3a6bda8581c142ec0f5fc378c001825cdea974d6...            0   
14     001a014cbe44de55aba63758459189edae8baf93975b72...            0   
15     001a0249de4b9ceb64fd92651488f5fd1859853e7aef9d...            0   
16     001a732e2eb4c808f143827e1e6148fd0a58c77b221f0a...            0   
17     001ced2742f344f2d18865eec8e54e8213ea288584e397...            0   
18     001f9c89eabdab1f5a66c14a1b8df8e1032328b7e0deda...            0   
19     0020006d31ab79ccf6cbf04e46fcfec256a4df2f05edf1...            0   
20     002049a63d242fea89bc61a55f4e1dd8260b97d19ff439...            0   
21     002080e409266fe03b7d21a0bed6f34ba8cc685215708f...            0   
22     0020d3aa43ce4bfb0434a1aa6e8e6dda26d7983e8f2c3c...            0   
23     002213b34d80bb2b262e116e8cc5453b52e1f9ba20ef2e...            0   
24     002388224decb6a748902df4c828207315d77c1d882a07...            0   
25     0024bee71339308003fdd0e18005e4476014858c8083bf...            0   
26     0024e806b932edae72e0410633c5b90dd54538166f858a...            0   
27     00257e2ab3eca2a8240d1ba6434e6d72e1094d59559fa2...            0   
28     002645d15c805ac9d30ccfd9488ea3891e5da50aa9c5ef...            0   
29     00285603fb876fe1ed8aba04c61e8de27907ce1c37effd...            0   
...                                                  ...          ...   
49970  ffd2bf6c1030a80ce08032d26cf906d23180399707c03f...            0   
49971  ffd31614c0c3e3fd0a66465835118bfb77553df467a924...            0   
49972  ffd43074f4323ab700c009e7b4c2b5c28071d5ea7fea64...            0   
49973  ffd87c1ab4b92268c1f405a64f2f1c3b14da8105bb415d...            0   
49974  ffd8cdfa3719ce7172bb1432692302066643c12c3ede18...            0   
49975  ffdae2c18492dee63616c25e575aa8c2622adce61deb5c...            0   
49976  ffdb360e4d84935ce579c66c7a6f4eaf0628986d158842...            0   
49977  ffdbb7636bfd1bd0b838d0f1b40601b6a80c0f0134bff3...            0   
49978  ffdc372079293328bfd9bdfc08b658dc8b9ff74225b412...            0   
49979  ffdcf82d61e4076b953f4c2497e6dc30bd83440688866c...            0   
49980  ffde7d94a0ec5c4994e5d0b745595e8e19e28de3143aef...            0   
49981  ffdf612a724a5017994a49e63f1d3eacb7779e8d4dd21e...            1   
49982  ffe2b61c5774852688e806ae9c133f4c90d346d4908a28...            0   
49983  ffe2d17a4c4184e9bcf202cf30b63a38c66549c4a4f98e...            0   
49984  ffe32871f239edec0393ef4fb7505aab1b0fd5e23fe51b...            0   
49985  ffe4083dcfceb88183cbe4daba360f66777b2ae1a10aa8...            0   
49986  ffe66b3310890bdc1b369124fd2fe64c623e8bc3dac4a5...            0   
49987  ffe7dd4744fecd97a8448066f18b1d4002cb61b6f8906f...            0   
49988  ffeb215ade5e7e7c9003148aed8fe670c2e39103bf8ca8...            0   
49989  fff16b8229d093ec253f17268926d5694dd05df89c3ed6...            0   
49990  fff1ae554221f2fad5187281df1714336b29fec9bf33e7...            0   
49991  fff1ed854e3ebb2eaddd4a7e2c83b39b292408944d0222...            0   
49992  fff704275847c12f541e6fab3cb55f422f6237b64aca1f...            0   
49993  fff749604ce51e9c9bea33ea0e

In [17]:
data_export.to_csv(r'/home/cloudera-scm/Machine_learning/telenor_hackathon/hackathon/processed_hackathon_prepost.csv', sep=',')